In [2]:
import argparse
import pandas as pd
import numpy as np
import selenium

In [3]:
# Spotify API credentials from your Spotify Developer Dashboard
client_id = '4c1940f4e36240a3b2ab4ca96df436d5'
client_secret = 'd6878abd48cf457d9e1d69502b3ab46f'

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [5]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

username = "marcos.micheloni@gmail.com"

In [6]:
def link_to_uri (link):
    return link.split("/")[-1].split("?")[0]

In [7]:
sp.audio_features(link_to_uri("https://open.spotify.com/track/6Vjxx2oGhC1jS5zkeIjoSX?si=20635e600ba34e3d"))

[{'danceability': 0.63,
  'energy': 0.5,
  'key': 2,
  'loudness': -14.953,
  'mode': 1,
  'speechiness': 0.0337,
  'acousticness': 0.29,
  'instrumentalness': 7.63e-06,
  'liveness': 0.133,
  'valence': 0.526,
  'tempo': 139.783,
  'type': 'audio_features',
  'id': '6Vjxx2oGhC1jS5zkeIjoSX',
  'uri': 'spotify:track:6Vjxx2oGhC1jS5zkeIjoSX',
  'track_href': 'https://api.spotify.com/v1/tracks/6Vjxx2oGhC1jS5zkeIjoSX',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6Vjxx2oGhC1jS5zkeIjoSX',
  'duration_ms': 194027,
  'time_signature': 4}]

In [9]:
# dictionary containing links to every playlist for each year
year_link_dict = {
    2012: "https://open.spotify.com/playlist/06q0ylRXnaaxNntx6DfMmb?si=26cb0da95c8c4038&pt=45ce7953c234a41fef3e49dfab2e82de",
    2013: "https://open.spotify.com/playlist/7A2y24vTcfs2PeHc3YkjYe?si=cd22b32301ac404a",
    2014: "https://open.spotify.com/playlist/4p0LLXmQk4WOhpjGQ16eW7?si=eb652a4d4349461e&pt=e862d0af6633d6386b7ca0212d8efdb4",
    2015: "https://open.spotify.com/playlist/5j8mBlHCoqdhKS3NwnjdzJ?si=b9e004e52ab4471d",
    2016: "https://open.spotify.com/playlist/08kp8Sr0UnrLRkafdNrUCM?si=c02f061a24b7499c&pt=e53d60ef50320a02f8a09d1424aa89fe",
    2017: "https://open.spotify.com/playlist/7hvR06ICC2g29uJ3xCz3hU?si=26f3a50c993d4937",
    2018: "https://open.spotify.com/playlist/7n4Gr817HIJTRozue2awwu?si=7837db52993745fc&pt=b944bae48f027ac0ded6abbdf3564e4f",
    2019: "https://open.spotify.com/playlist/6MwUs5ffwsQqCGnivhCUJa?si=64d939a35b6e420e",
    2020: "https://open.spotify.com/playlist/2aFTa55ZL8vGo09chJO8o1?si=6bfc8c3a205d4905&pt=5ee0cc2bfe60cca5ca93078701e6b470",
    2021: "https://open.spotify.com/playlist/4eGzTH6CxuZTPODzMMPWbU?si=1e6da205723d43fc",
    2022: "https://open.spotify.com/playlist/6j2F1qMu5RUXEEukGjGNKe?si=f547c7df518145a6&pt=6ff3bfc91adb3bee970923871ed8e692"
    }

In [10]:
list_of_series = []
cols = ["year", "rank", "name", "artist", "uri", "danceability", "energy", "key", "loudness", "mode",
        "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration",
        "artist_genres", "artist_popularity", "artist_followers", "playlist"]


for year, link in year_link_dict.items():
    tracks = sp.playlist_tracks(link)["items"]
    
    for index, track in enumerate(tracks):

        track_name = track["track"]["name"]
        artist_name = track["track"]["artists"][0]["name"]
        artist_id = track["track"]["artists"][0]["id"]
        track_uri = track["track"]["uri"]
        track_rank = index+1

        audio_features = sp.audio_features(track_uri)[0]

        track_danceability = audio_features["danceability"]
        track_energy = audio_features["energy"]
        track_key = audio_features["key"]
        track_loudness = audio_features["loudness"]
        track_mode = audio_features["mode"]
        track_speechiness = audio_features["speechiness"]
        track_acousticness = audio_features["acousticness"]
        track_instrumentalness = audio_features["instrumentalness"]
        track_liveness = audio_features["liveness"]
        track_valence = audio_features["valence"]
        track_tempo = audio_features["tempo"]
        track_duration = audio_features["duration_ms"]

        artist_data = sp._get("artists/"+artist_id)
        if artist_data:
            artist_genres = artist_data["genres"]
            artist_popularity = artist_data["popularity"]
            artist_followers = artist_data["followers"]["total"]
        else:
            artist_genres = np.nan
            artist_popularity = np.nan
            artist_followers = np.nan
        
        list_of_series.append(pd.Series([year, track_rank, track_name, artist_name, track_uri, track_danceability,
                                         track_energy, track_key, track_loudness, track_mode,
                                         track_speechiness, track_acousticness, track_instrumentalness,
                                         track_liveness, track_valence, track_tempo, track_duration,
                                         artist_genres, artist_popularity, artist_followers, link],
                                         index=cols))

df = pd.DataFrame(list_of_series, columns=cols)

In [11]:
df.head()

,year,rank,name,artist,uri,danceability,energy,key,loudness,mode,...,acousticness,instrumentalness,liveness,valence,tempo,duration,artist_genres,artist_popularity,artist_followers,playlist
0,2012,1,Somebody That I Used To Know,Gotye,spotify:track:4wCmqSrbyCgxEXROQE6vtV,0.864,0.495,0,-7.036,1,...,0.5910,0.000133,0.0992,0.720,129.062,244973,[australian pop],65,2323179,https://open.spotify.com/playlist/06q0ylRXnaax...
1,2012,2,Call Me Maybe,Carly Rae Jepsen,spotify:track:3TGRqZ0a2l1LRblBkJoaDx,0.783,0.580,7,-6.548,1,...,0.0114,0.000002,0.1080,0.660,120.021,193400,"[canadian pop, dance pop, pop]",69,5173616,https://open.spotify.com/playlist/06q0ylRXnaax...
2,2012,3,We Are Young (feat. Janelle Monáe),fun.,spotify:track:7a86XRg84qjasly9f6bPSD,0.378,0.638,10,-5.576,1,...,0.0200,0.000077,0.0849,0.735,184.086,250627,"[baroque pop, metropopolis]",64,3418456,https://open.spotify.com/playlist/06q0ylRXnaax...
3,2012,4,Payphone,Maroon 5,spotify:track:1LmN9SSHISbtp9LoaR5ZVJ,0.739,0.756,4,-4.828,1,...,0.0136,0.000000,0.3700,0.523,110.028,231387,[pop],82,41729321,https://open.spotify.com/playlist/06q0ylRXnaax...
4,2012,5,Lights,Ellie Goulding,spotify:track:11ozIUBoXAgRTVWelDn4pL,0.703,0.731,8,-6.283,0,...,0.0721,0.001910,0.1380,0.719,120.028,244267,"[indietronica, metropopolis, pop, uk pop]",77,12380596,https://open.spotify.com/playlist/06q0ylRXnaax...


In [12]:
#df.to_csv("Top 50 Songs.csv", sep = ",")
df.to_csv("Top 50 Songs.csv", sep = ",")